In [16]:
import numpy as np
import pandas as pd
import pgeocode as pg
from utils import listdir_nohidden, assign_types
import os

nomi = pg.Nominatim('gb')

path = 'raw_gp/'

month_folders = listdir_nohidden(path) # path to downloaded data

addresses = []
gp_datas = []

for month in month_folders:
    address = pd.read_csv(path+month+'/Address.csv',
                          usecols=['PracticeId', 'Postcode'])

    gp_path = [f for f in os.listdir(path+month) if f.startswith("GPData")][0]
    gp_data = pd.read_csv(path+month+'/'+gp_path, 
                          usecols=['Period', 'BNFCode', 'PracticeID', 'Quantity', 'DDD'])
    
    gp_data['Quantity/DDD'] = gp_data['Quantity']/gp_data['DDD']

    gp_data = assign_types(gp_data)

    zips = address['Postcode'].apply(lambda x: x[:4]) # get 1st 4 chars of postcode
    geo_query = nomi.query_postal_code(zips.to_numpy()) # lookup coords from postcodes

    # Add coords
    address['lat'] = geo_query['latitude']
    address['long'] = geo_query['longitude']
    
    addresses.append(address)
    gp_datas.append(gp_data)

    print(gp_path+' done!')

addresses = pd.concat(addresses, axis=0)
gp_datas = pd.concat(gp_datas, axis=0)

GPData202006.csv done!
GPData202202.csv done!
GPData202205.csv done!
GPData202008.csv done!
GPData202204.csv done!
GPData202009.csv done!
GPData202203.csv done!
GPData202007.csv done!
GPData202110.csv done!
GPData201806.csv done!
GPData201906.csv done!
GPData202111.csv done!
GPData202301.csv done!
GPData202102.csv done!
GPData202105.csv done!
GPData201809.csv done!
GPData202104.csv done!
GPData201909.csv done!
GPData202103.csv done!
GPData201903.csv done!
GPData202210.csv done!
GPData201902.csv done!
GPData201910.csv done!
GPData202012.csv done!
GPData202002.csv done!
GPData202211.csv done!
GPData201810.csv done!
GPData202005.csv done!
GPData202208.csv done!
GPData202206.csv done!
GPData202201.csv done!
GPData201911.csv done!
GPData201912.csv done!
GPData201807.csv done!
GPData201811.csv done!
GPData202207.csv done!
GPData202003.csv done!
GPData202004.csv done!
GPData202209.csv done!
GPData201907.csv done!
GPData201812.csv done!
GPData201904.csv done!
GPData201901.csv done!
GPData20180

In [17]:
merged = pd.merge(gp_datas, addresses, 
                  left_on='PracticeID',
                  right_on='PracticeId',
                  how='left').drop(columns='PracticeId')

In [18]:
print(merged['Quantity/DDD'].isnull().sum())

merged.head()

0


,PracticeID,BNFCode,Quantity,DDD,Period,Quantity/DDD,type,Postcode,lat,long
0,W00118,0501011P0AAAJAJ,56,7.0,202006,8.0,penicillins,NP4 0TB,51.718516,-3.030521
1,W00118,0501011P0AAAJAJ,56,7.0,202006,8.0,penicillins,NP4 0TB,51.718516,-3.030521
2,W00118,0501011P0AAAJAJ,56,7.0,202006,8.0,penicillins,NP4 0TB,51.718516,-3.030521
3,W00118,0501011P0AAAJAJ,56,7.0,202006,8.0,penicillins,NP4 0TB,51.718516,-3.030521
4,W00118,0501011P0AAAJAJ,56,7.0,202006,8.0,penicillins,NP4 0TB,51.718516,-3.030521


In [19]:
merged.to_csv('FilteredGPData.csv.zip', index=False, compression='gzip')